# **Image Super-Resolution**
**64x64 -> 512x512**

A colab notebook for upscaling 64x64 images to 512x512, using [this](https://github.com/Janspiry/Image-Super-Resolution-via-Iterative-Refinement).


## Setup
---
Instructions:

*   Turn on hardware acceleration under `Runtime -> Change Runtime Type -> Hardware accelerator -> GPU`
*   Use this command to ensure that the connected machine has a GPU:

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

*    Now, execute each cell sequentially, waiting until each one is done before running the next cell.


### Clone repo, download a pre-trained model, install dependencies.


In [ ]:
%cd /content/
!git clone https://github.com/Janspiry/Image-Super-Resolution-via-Iterative-Refinement.git
!pip install tensorboardx

**NOTE: The `gdown` command does not always work.** If it gives you an error, as a temporary workaround, you must manually upload files for the pre-trained model. Download the files [here](https://drive.google.com/drive/folders/1mCiWhFqHyjt5zE4IdA41fjFwCYdqDzSF) (you may have to log into your google account to be able to do so). Then, click the `files` button on the left hand side of the screen, and find the folder labled `Image-Super-Resolution-via-Iterative-Refinement`. Click the three dots on the right of the folder, and click `upload`. Now, select the model files that you downloaded.

**ALTERNATIVELY: if you plan to use this script frequently, you can make a copy of the model files in your own google drive.** Go [here](https://drive.google.com/drive/folders/1mCiWhFqHyjt5zE4IdA41fjFwCYdqDzSF), right click on each file, and hit `Make a copy`. Next, locate the files in your own google drive, and rename them from `Copy of I830000_E32_opt.pth` and `Copy of I830000_E32_gen.pth` to `I830000_E32_opt.pth` and `I830000_E32_gen.pth`. Next, right click on each file, and hit `Get link`. Click on `Restricted`, and change it to `Anyone with the link...`, as `Viewer`. Now, copy the links. They should look something like this: `https://drive.google.com/file/d/<file id>/view?usp=sharing` - copy the ids, and paste them into the `gdown` commands below.  

For some reason, the model files are getting rate limited at times. This will hopefully be fixed in the future.

In [ ]:
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/
!gdown --id 1M015uQG8cq0mqwhEfc7klFJwDr2jsYSr
!gdown --id 1Vwe1Raa-Dpop75o5v7O5qvGoOdFDlyc-
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/config/

### Patch config files

Create patchfile



In [ ]:
%%writefile 64_512.patch
--- sr_sr3_64_512_new.json	2021-10-22 16:20:20.901133618 +0000
+++ sr_sr3_64_512.json	2021-10-22 16:20:52.036081672 +0000
@@ -9,8 +9,8 @@
         "tb_logger": "tb_logger",
         "results": "results",
         "checkpoint": "checkpoint",
-        "resume_state": null
-        // "resume_state": "experiments/distributed_high_sr_ffhq_210901_121212/checkpoint/I830000_E32" //pretrain model or training state
+        // "resume_state": null
+        "resume_state": "I830000_E32" //pretrain model or training state
     },
     "datasets": {
         "train": {



Apply patchfile

In [ ]:
!apt-get install dos2unix
!dos2unix sr_sr3_64_512.json
!patch < 64_512.patch

### Prepair Data
Upload 64x64 pixel image(s) to be upscaled

*Click the **browse** button and select the images you would like to upscale.*



In [ ]:
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/
!mkdir -p input
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/input/
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  savefile = open(fn, 'wb')
  savefile.write(uploaded[fn])
  print('Successfully uploaded "{}" ({} bytes).'.format(fn, len(uploaded[fn])))
  savefile.close()

Generate neccesary files and directory structure to begin upscaling.  

In [ ]:
!pip install lmdb

In [ ]:
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/
!python data/prepare_data.py --path /content/Image-Super-Resolution-via-Iterative-Refinement/input/ --size 64,512 --out ./dataset/celebahq

## Upscaling
---
Note that this may take ~30 minutes or more, depending on which GPU you get.

Output images will be located under ./experiments/\<some folder\>/results

In [ ]:
!python infer.py -c /content/Image-Super-Resolution-via-Iterative-Refinement/config/sr_sr3_64_512.json